<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Slave_Voyages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/slavevoyages/SlaveVoyages.png?raw=true" />

**Comércio Transatlântico de Escravos**

O site Slave Voyages, documentou e disponibilizou de forma digital, arquivos sobre esse triste período da história mundial e, em especial, para o Brasil, onde o tráfico negreiro perdurou até a criação da Lei Áurea de 1888. Existem registros de mercado de escravos, desde 1444 em Portugal. E existem registros bem antigos dessa prática, ainda em impérimos como o dos Romanos, Gregos, Árabes.  

### Sobre os dados

  * Explorar a disperssão de Africanos escravizados pelo Atlântico
    * https://www.slavevoyages.org/
    * Dataset: https://www.slavevoyages.org/voyage/database 

## Obtenção e transformação dos dados

In [18]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# importando dados
dataset = "https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/slavevoyages/data_SlaveVoyages.csv"
df = pd.read_csv(dataset)

# entredas e variáveis
print('Entradas: {}' .format(df.shape[0]))
print('Variáveis: {}' .format(df.shape[1]))

Entradas: 36110
Variáveis: 8


In [20]:
# carregando dataset
df.head()

,Identidade da viagem,Vessel name,Voyage itinerary imputed port where began (ptdepimp) place,Voyage itinerary imputed principal place of slave purchase (mjbyptimp),Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place,Year of arrival at port of disembarkation,Slaves arrived at 1st port,Captain's name
0,10354,Wakende Kraan,Hellevoetsluis,Elmina,Curacao,1706,177.0,"Hoeven, Jan van der<br/> Wenman, Roelof"
1,10355,Wapen van Amsterdam,NaN,Ardra,NaN,1685,NaN,NaN
2,10356,Wapen van Amsterdam,Texel,"Gold Coast, port unspecified",Curacao,1688,NaN,"Scheij, Pieter"
3,10357,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Curacao,1701,459.0,"Stoop, Pieter<br/> Crans, Pieter"
4,10358,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Suriname,1702,386.0,"Crans, Pieter"


In [ ]:
df[]

Para falicitar nas análises, vamos diminuir o tamanho do nome das columas e também, trazer para o português. Vamos criar um novo Dataset e manter o original, que servirá para consultas futuras.

In [22]:
# Dataset redução e tradução de colunas
df_traducao = df.rename(columns={'Vessel name': 'Nome Embarcação', 
                                 'Voyage itinerary imputed port where began (ptdepimp) place':'Porto onde viagem começou',
                                 'Voyage itinerary imputed principal place of slave purchase (mjbyptimp) ':'Local de aquisição',
                                 'Voyage itinerary imputed principal port of slave disembarkation (mjslptimp) place': 'Local Desembarque',
                                 'Year of arrival at port of disembarkation': 'Ano Chegada',
                                 'Slaves arrived at 1st port':'Qtde.',
                                 "Captain's name":'Nome Capitão'})


# carregando dataset
df_traducao.head()

,Identidade da viagem,Nome Embarcação,Porto onde viagem começou,Local de aquisição,Local Desembarque,Ano Chegada,Qtde.,Nome Capitão
0,10354,Wakende Kraan,Hellevoetsluis,Elmina,Curacao,1706,177.0,"Hoeven, Jan van der<br/> Wenman, Roelof"
1,10355,Wapen van Amsterdam,NaN,Ardra,NaN,1685,NaN,NaN
2,10356,Wapen van Amsterdam,Texel,"Gold Coast, port unspecified",Curacao,1688,NaN,"Scheij, Pieter"
3,10357,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Curacao,1701,459.0,"Stoop, Pieter<br/> Crans, Pieter"
4,10358,Wapen van Holland (a) Hollandia,Hellevoetsluis,Whydah,Suriname,1702,386.0,"Crans, Pieter"
